In [88]:
import os
import numpy as np
from os import listdir
from PIL import Image
import time
import tensorflow as tf
from tensorflow.keras import layers,models,optimizers
from keras import backend as K
import matplotlib.pyplot as plt

In [2]:
path="C:/DL_Dataset/Project_data"

In [172]:
def load_npdata(path):
    for r, d, f in os.walk(path):
        direc=d
        break
    rs = np.random.RandomState(123)
    rs.shuffle(direc)
    direc = direc[:len(direc)-500]
    
    np_images=[]
    start= time.time()
    for d in direc:
        dic={}
        sub=[a for a in listdir(path+"/"+d)]
        flag=0
        for ele in sub:
            mypath = path+"/"+d+"/"+ele+"/"
            onlyfiles = [mypath+f for f in listdir(mypath)]
            temp=[]
            for addr in onlyfiles:
#                 temp.append(np.array(Image.open(addr)))
                original_img = np.array(Image.open(addr))
                downscaled_img = tf.compat.v1.image.resize_image_with_pad(original_img,32,32)
                temp.append(downscaled_img)
            if len(temp)>0:
                if ele[0].lower()=='f':
                    dic['father']=temp
                elif ele[0].lower()=='m':
                    dic['mother']=temp
                elif ele.lower()=='child_male':
                    dic['child']=temp    
                    dic['gender']=np.zeros((temp[0].shape))
                elif ele.lower()=='child_female':
                    dic['child']=temp    
                    dic['gender']=np.ones((temp[0].shape))    
            else:
                flag=1
                break

        if flag!=1:
            for x in dic['father']:
                for y in dic['mother']:
                    for z in dic['child']:
                        np_images.append([x,y,dic['gender'],z])
    print("time:",time.time()-start)
    return np_images

In [173]:
full_data = load_npdata(path)
print(len(full_data))

time: 117.30267071723938
1089290


create encoder decoder supervised pipeling operation
input: stacked images of father and mother (32,32,6)
encoder -> decoder -> image reconstruction
reconstruction loss w/ target image as child

In [76]:
concat = tf.keras.layers.Concatenate()
inp = concat([full_data[0][0],full_data[0][1]])
child = full_data[0][3]
inp = tf.convert_to_tensor(inp, dtype=tf.float32)

In [69]:
OUTPUT_CHANNELS = 3

In [70]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [71]:
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [109]:
def EncoderNN():
  down_stack = [
    downsample(3, 4, apply_batchnorm=True),
    downsample(3, 4, apply_batchnorm=False),
    downsample(3, 4, apply_batchnorm=False) 
  ]

  up_stack = [
    upsample(3, 4, apply_dropout=True),
    upsample(3, 4, apply_dropout=False),
    upsample(3, 4, apply_dropout=False)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh') # (bs, 256, 256, 3)

  concat = tf.keras.layers.Concatenate()

  inputs = tf.keras.layers.Input(shape=[32,32,6])
  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = concat([x, skip])
    
  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [163]:
encoder = EncoderNN()

# gen_output = encoder(inp[tf.newaxis,...], training=False)
# loss= tf.reduce_mean(tf.image.total_variation(tf.concat([child,tf.squeeze(gen_output)],axis=0)))
# loss = tf.reduce_mean(tf.nn.l2_loss(child-gen_output))
# print(loss)

In [117]:
def custom_loss():
    def loss(y_true,y_pred):
         print(y_true.shape)
         print(y_pred.shape)
         return tf.reduce_mean(tf.image.total_variation(tf.concat([y_true,y_pred],axis=0)))
    return loss

inp = concat([full_data[0][0],full_data[0][1]])
inp = tf.convert_to_tensor(inp, dtype=tf.float32)
X_train = tf.reshape(inp,([1,32,32,6]))
Y_train = tf.reshape(full_data[0][3],(1,32,32,3))

encoder.compile(loss=custom_loss(),
              optimizer=tf.keras.optimizers.RMSprop())
history = encoder.fit(X_train, Y_train,
                    batch_size=1,
                    epochs=1)

gen_output = encoder(inp[tf.newaxis,...], training=False)

In [151]:
concat = tf.keras.layers.Concatenate()

In [174]:
encoder.compile(loss=custom_loss(),optimizer=tf.keras.optimizers.RMSprop())

divide = 50000
batch = 1000
epoch = 5
for i in range(len(full_data)//divide):
    test= np.asarray(full_data[i*divide:(i+1)*divide])
    print("Data divided")
    X_train = tf.convert_to_tensor(concat([test[:,0],test[:,1]]),dtype =tf.float32)
    print("Batch converted to tensor")
    Y_train = test[:,3]
    history = encoder.fit(X_train, Y_train,batch_size=batch,epochs=epoch)
    break

(None, None, None, None)
(None, 32, 32, 3)


ValueError: setting an array element with a sequence.